In [2]:
with open('text.txt', 'r', encoding="utf-8") as f:
    text = f.read()

In [3]:
print('lenth of dataset in characters: ', len(text))

lenth of dataset in characters:  11853750


In [4]:
#Printing first 100 characters
print(text[:100])

Albert of Prussia ( 17 May 1490 20 March 1568 ) was the last Grand Master of the Teutonic Knights , 


In [5]:
#All unique chr in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz{
86


In [6]:
#mapping of char to int
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # str to int list
decode = lambda l: ''.join([itos[i] for i in l]) #int to str

print(encode("Hello"))
print(decode(encode("Hello")))

[40, 63, 70, 70, 73]
Hello


In [7]:
#Encoding entire text in dataset and storing in a torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print (data.shape,data.type)
print(data[:100])

torch.Size([11853750]) <built-in method type of Tensor object at 0x000002D4AFE730B0>
tensor([33, 70, 60, 63, 76, 78,  1, 73, 64,  1, 48, 76, 79, 77, 77, 67, 59,  1,
         8,  1, 17, 23,  1, 45, 59, 83,  1, 17, 20, 25, 16,  1, 18, 16,  1, 45,
        59, 76, 61, 66,  1, 17, 21, 22, 24,  1,  9,  1, 81, 59, 77,  1, 78, 66,
        63,  1, 70, 59, 77, 78,  1, 39, 76, 59, 72, 62,  1, 45, 59, 77, 78, 63,
        76,  1, 73, 64,  1, 78, 66, 63,  1, 52, 63, 79, 78, 73, 72, 67, 61,  1,
        43, 72, 67, 65, 66, 78, 77,  1, 12,  1])


In [ ]:
#split of data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]